In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import time
import random
import re
import pandas as pd
import feather
from sklearn import datasets
import sklearn

In [2]:
def separate_dataset(trainset, ratio = 0.5):
    datastring = []
    datatarget = []
    for i in range(len(trainset.data)):
        data_ = trainset.data[i].split('\n')
        data_ = list(filter(None, data_))
        data_ = random.sample(data_, int(len(data_) * ratio))
        for n in range(len(data_)):
            data_[n] = clearstring(data_[n])
        datastring += data_
        for n in range(len(data_)):
            datatarget.append(trainset.target[i])
    return datastring, datatarget

def clearstring(string):
    string = re.sub('[^A-Za-z0-9 ]+', '', string)
    string = string.split(' ')
    string = filter(None, string)
    string = ' '.join(string)
    return string.lower()

trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)

train = pd.DataFrame(trainset.data,columns=['sequence'])
label = pd.DataFrame(trainset.target,columns=['labels'])

In [3]:
num_words = 50000
column_name = 'word_seg'
word_seq_len = 1800
embedding_vector = 200

In [6]:
from keras.preprocessing import text,sequence
from gensim.models import Word2Vec
num_words = 50000
import os
import numpy as np

def w2v_pad(df_train, col, maxlen_, victor_size):
    tokenizer = text.Tokenizer()
    tokenizer.fit_on_texts(list(df_train[col].values))
    
    train_ = sequence.pad_sequences(tokenizer.texts_to_sequences(df_train[col].values), maxlen=maxlen_)
    word_index = tokenizer.word_index
    
    count = 0
    nb_words = len(word_index)
    print(nb_words)
    
    all_data = df_train[col]
    file_name = './Word2Vec_' + col  +"_"+ '200' + '.model'
    if not os.path.exists(file_name):
        model = Word2Vec([[word for word in document.split(' ')] for document in all_data.values],
                         size=victor_size, window=5, iter=10, workers=11, seed=2018, min_count=2)
        model.save(file_name)
    else:
        model = Word2Vec.load(file_name)
        
    embedding_word2vec_matrix = np.zeros((nb_words+1, victor_size))
    for word, i in word_index.items():
        embedding_vector = model[word] if word in model else None
        if embedding_vector is not None:
            count += 1
            embedding_word2vec_matrix[i] = embedding_vector
        else:
            unk_vec = np.random.random(victor_size) * 0.5
            unk_vec = unk_vec - unk_vec.mean()
            embedding_word2vec_matrix[i] = unk_vec    
    
    return train_, word_index, embedding_word2vec_matrix

In [7]:
train_, word2idx, word_embedding = w2v_pad(train, 'sequence', 40, 200)

20465


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [9]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

lb = LabelEncoder()
train_label = lb.fit_transform(label['labels'].values)
train_label = to_categorical(train_label)

In [19]:
class Model:
    def __init__(self, size_layer, num_layers, embedding_matrix, 
                 dimension_output, learning_rate):
        def cells(size=128, reuse = False):
            return tf.nn.rnn_cell.BasicLSTMCell(size, reuse=reuse)
        
        def add_highway(x, i):
            reshaped = tf.reshape(x, [-1, 40*200])
            H = tf.layers.dense(reshaped, 40*200, tf.nn.relu)
            T = tf.layers.dense(reshaped, 40*200, tf.sigmoid)
            C = tf.subtract(tf.to_double(1.0), T)
            highway_out = tf.add(tf.multiply(H, T), tf.multiply(H, C))
            return tf.reshape(highway_out, [-1, 40, 200])
    
        self.X = tf.placeholder(tf.int32, [None, 40])
        self.Y = tf.placeholder(tf.float32, [None, dimension_output])
        
        encoder_embedding = tf.Variable(embedding_matrix)
        encoder_embedded = tf.nn.embedding_lookup(encoder_embedding, self.X)
        w1 = tf.get_variable("w1", [3, 200, 200],dtype=tf.float64)
        b1 = tf.get_variable("b1", [1, 1, 2], dtype=tf.float64)
        out = tf.nn.conv1d(encoder_embedded, w1, stride=1, padding="SAME")
        
        out = add_highway(out, 1)

        rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)])
        drop = tf.contrib.rnn.DropoutWrapper(rnn_cells, output_keep_prob=0.5)
        outputs, _ = tf.nn.dynamic_rnn(drop, out, dtype=tf.float64)
        
        W = tf.get_variable('w',shape=(128, dimension_output),dtype=tf.float64,initializer=tf.orthogonal_initializer())
        b = tf.get_variable('b',shape=(dimension_output),dtype=tf.float64,initializer=tf.zeros_initializer())
        self.logits = tf.matmul(outputs[:, -1], W) + b
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [20]:
size_layer = 128
num_layers = 1
embedded_size = 128
dimension_output = len(trainset.target_names)
learning_rate = 1e-3
maxlen = 50
batch_size = 128

In [21]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(size_layer,num_layers,word_embedding,dimension_output,learning_rate)
sess.run(tf.global_variables_initializer())

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Instructions for updating:
Use tf.cast instead.


In [22]:
train_X, test_X, train_Y, test_Y = train_test_split(train_, train_label, test_size = 0.2)

In [23]:
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 5, 0, 0, 0
while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n'%(EPOCH))
        break
        
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    for i in range(0, (len(train_X) // batch_size) * batch_size, batch_size):
        batch_x = train_X[i:i+batch_size]
        batch_y = train_Y[i:i+batch_size]
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                           feed_dict = {model.X : batch_x, model.Y : batch_y})
        train_loss += loss
        train_acc += acc
    
    for i in range(0, (len(test_X) // batch_size) * batch_size, batch_size):
        batch_x = test_X[i:i+batch_size]
        batch_y = test_Y[i:i+batch_size]
        acc, loss = sess.run([model.accuracy, model.cost], 
                           feed_dict = {model.X : batch_x, model.Y : batch_y})
        test_loss += loss
        test_acc += acc
    
    train_loss /= (len(train_X) // batch_size)
    train_acc /= (len(train_X) // batch_size)
    test_loss /= (len(test_X) // batch_size)
    test_acc /= (len(test_X) // batch_size)
    
    if test_acc > CURRENT_ACC:
        print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time()-lasttime)
    print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
    EPOCH += 1

epoch: 0, pass acc: 0.000000, current acc: 0.585449
time taken: 8.406481504440308
epoch: 0, training loss: 0.691094, training acc: 0.556226, valid loss: 0.670667, valid acc: 0.585449

epoch: 1, pass acc: 0.585449, current acc: 0.679688
time taken: 5.805678367614746
epoch: 1, training loss: 0.615616, training acc: 0.660156, valid loss: 0.593044, valid acc: 0.679688

epoch: 2, pass acc: 0.679688, current acc: 0.756836
time taken: 5.724946975708008
epoch: 2, training loss: 0.359799, training acc: 0.842093, valid loss: 0.600550, valid acc: 0.756836

time taken: 5.990009546279907
epoch: 3, training loss: 0.209559, training acc: 0.917495, valid loss: 0.840374, valid acc: 0.711914

time taken: 6.08944296836853
epoch: 4, training loss: 0.120347, training acc: 0.953007, valid loss: 0.908999, valid acc: 0.740723

time taken: 5.823898792266846
epoch: 5, training loss: 0.088830, training acc: 0.964252, valid loss: 1.212389, valid acc: 0.725098

time taken: 5.847049236297607
epoch: 6, training loss